# ĐỒ ÁN CUỐI KỲ CỦA NHÓM 13: 
# DỰ ĐOÁN GIÁ TẦM ĐIỆN THOẠI Ở VIỆT NAM

# 1. Thu Thập dữ liệu

In [2]:
# Thư viện cần thiết
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import pandas as pd
import csv
import math
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram')
import sklearn

## a. Thu thập link điện thoại:

Chúng ta sẽ crawl dữ liệu điện thoại hiện hành đang bán ở Việt Nam trên trang ***'https://www.dienmayxanh.com/dien-thoai'***

**Chúng em đã check file robots.txt của trang là nhận thấy việc crawl dữ liệu điện thoại là được cho phép**
 - Với cột Giá Điện Thoại, chúng ta không lấy giá bán sau khi đã khuyến mại mà lấy giá niêm yết của sản phẩm khi được phân phối ở thị trường Việt Nam 
   => Dữ liệu về giá sẽ khách quan hơn vì nếu lấy giá khuyến mãi thì mỗi cửa hàng sẽ có một giá bán khác nhau.

**NOTE:** Do dữ liệu biến động theo trang web (có thể có thêm điện thoại mới) nên lúc thầy test có thể dữ liệu sẽ khác hơn so với chúng em ạ

In [3]:
def collect_phone_links(link):
    # Tương tác với trang web, bấm nút xem thêm cho đến khi hết trang web (khoảng 7 lần bấm)
    browser = webdriver.Chrome(executable_path='./chromedriver.exe') 
    browser.get(link)
    i = 0
    while i < 7:
        browser.find_element_by_class_name('loadmore').click()
        i = i + 1
    html_text = browser.page_source
    browser.quit()
    # parse html để lấy phone links
    phone_link_list = []
    parse_html = BeautifulSoup(html_text, 'html.parser')
    phones = parse_html.find_all('a', {'class': 'prdImageHrf'})
    for phone in phones:
        phone_link = 'https://www.dienmayxanh.com/' + phone['href']
        phone_link_list.append(phone_link)
    return phone_link_list
    

In [4]:
def write_links_into_file(link):
    data_link = open('phone_links.txt','w')
    links =  collect_phone_links(link)
    for link in links:
        data_link.write(link + '\n')
    data_link.close()

In [5]:
start_link = 'https://www.dienmayxanh.com/dien-thoai'
write_links_into_file(start_link)

Sau khi đã lấy được tất cả các phone link và lưu vào file phone_links.txt để dùng về sau.

In [6]:
phone_links = open('phone_links.txt','r')
raw_links  = phone_links.readlines()
phone_links.close()
links = []
for link in raw_links:
    link = link[:-1]
    links.append(link)

In [7]:
print('Tổng số phone link ta có: ',len(links))

Tổng số phone link ta có:  179


## b. Thu thập dữ liệu điện thoại:

In [8]:
def get_phone_infor(links):
    infor = []
    for link in links:
        get_html = requests.get(link).text
        parse_html = BeautifulSoup(get_html, 'html.parser')
        
        name = re.search(r'(?<=dien-thoai/)[\w-]+', link)
        get_phone_name =  name.group(0)
        #print(name)
        
        brand = re.search(r'(?<=dien-thoai/)[\w]+', link)
        get_brand = brand.group(0)
        
        tech_infor = parse_html.find('ul',{'class': 'specs'})
        if tech_infor!= None:
            get_tech_infor = tech_infor.text
        else:
            continue
        get_screen = re.search(r'(([\d.]+))"', get_tech_infor)
        if get_screen!= None:
            screen = get_screen.group(1)
        else:
            screen = np.nan
            
        get_OS = re.search(r'((?<=Hệ điều hành:)[\w ]+)([()\w ])*[A-Z]', get_tech_infor)
        if get_OS!= None:
            OS = get_OS.group(1)
        else:
            OS = np.nan
        
        get_main_camera = re.search(r'((?<=Camera sau:)([\w ]+)*[\d.]+) [MP&]', get_tech_infor)
        if get_main_camera!= None:
            main_camera = get_main_camera.group(1)
        else:
            main_camera = np.nan
            
        get_front_camera = re.search(r'((?<=Camera trước:)([\w ]+)*[\d.]+) MP', get_tech_infor)
        if get_front_camera!= None:
            front_camera = get_front_camera.group(1)
        else:
            front_camera = np.nan
            
        get_CPU =re.search(r'((?<=CPU:)[\w]+)', get_tech_infor)
        if get_CPU!= None:
            CPU = get_CPU.group(1)
        else:
            CPU = np.nan
        get_RAM = re.search(r'((?<=(RAM:))[\d]+) GB[Bộ]*', get_tech_infor)
        if get_RAM!= None:
            RAM = get_RAM.group(1)
        else:
            RAM = np.nan
            
        get_storage = re.search(r'((?<=Bộ nhớ trong:)[\d]+)[ GB]', get_tech_infor)
        if get_storage != None:
            storage = get_storage.group(1)
        else:
            storage = np.nan
        
        get_battery = re.search(r'((?<=Dung lượng pin:)[\w ]+)mAh', get_tech_infor)
        if get_battery!= None:
            battery = get_battery.group(1)
        else:
            battery = np.nan
            
        get_price = parse_html.find('strong',{'class': 'dl'})
        if get_price != None:
            price = get_price.text
        elif (parse_html.find('div',{'class': 'area_price'}) !=None) and (parse_html.find('span',{'class': 'nosell'}) == None) :
            get_price = parse_html.find('div',{'class': 'area_price'}).text
            price = re.search(r'([\d.]+)₫', get_price).group(1)
        else:
            price = np.nan
        #print(price)
        
        infor.append([get_phone_name,get_brand,screen,OS,main_camera,front_camera,CPU,RAM,storage,battery,price])
        
        
        time.sleep(1)
    return infor

In [9]:
data_phones = get_phone_infor(links)

In [11]:
df = pd.DataFrame(data = data_phones, columns = ["Phone_Name","Brand","Screen","OS","Main_Camera","Front_Camera","CPU","RAM","Storage","Battery",'Price'])
df.to_csv("Phone_Information.csv",index = False)
